In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
# Downgrade the version of scipy
pip install scipy==1.1.0

     |████████████████████████████████| 31.2MB 103kB/s 
ERROR: plotnine 0.6.0 has requirement scipy>=1.2.0, but you'll have scipy 1.1.0 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1


In [2]:
# import important libraries
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime

In [3]:
# Mount the drive to google colab notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Unzip the data
! unzip /content/drive/MyDrive/Data/Project_data.zip

Streaming output truncated to the last 5000 lines.
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00050.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00052.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00054.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00056.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00058.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00060.png  
 extracting: Project_data/train/WIN_20180926_17_34_37_Pro_Stop_new/WIN_20180926_17_34_37_Pro_00062.png  
   creating: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/
 extracting: Project_data/train/WIN_20180926_17_35_12_Pro_Thumbs_Down_new/WIN_20180926_17_35_12_Pro_00001.png  
 extracting: Project_data/t

We set the random seed so that the results don't vary drastically.

In [6]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation.

In [7]:
# Creating train and validation document by picking the data from folder with various 
#permutation to make sure that the data is not similar
train_doc = np.random.permutation(open('/content/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('/content/Project_data/val.csv').readlines())
batch_size = 32

# Generator

In [8]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,5,10,12,14,15,16,17,18,19,20,21,22,23,24]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches =len(t)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,15,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image =imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    if image.shape[1]==160:
                      image=imresize(image,(100,100))
                    else:
                        image=imresize(image,(100,100))

                    batch_data[folder,idx,:,:,0] = image[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do
        if (len(t) % batch_size != 0):
          batch_size = len(t) - (batch_size*num_batches)
          batch_data = np.zeros((batch_size,15,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
          batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
          for folder in range(batch_size): # iterate over the batch_size
            imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
            for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
              image =imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                      
              #crop the images and resize them. Note that the images are of 2 different shape 
              #and the conv3D will throw error if the inputs in a batch have different shapes
              if image.shape[1]==160:
                image=imresize(image,(100,100))
              else:
                image=imresize(image,(100,100))

              batch_data[folder,idx,:,:,0] = image[:,:,0] #normalise and feed in the image
              batch_data[folder,idx,:,:,1] = image[:,:,1] #normalise and feed in the image
              batch_data[folder,idx,:,:,2] = image[:,:,2] #normalise and feed in the image

          batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
        yield batch_data, batch_labels

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

# Get the current date and time to create h5 folder for different model executed 

In [28]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/Project_data/train'
val_path = '/content/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 30 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 30


# **Model**
Here you make the model using different functionalities that Keras provides. Remember to use Conv3D and MaxPooling3D and not Conv2D and Maxpooling2D for a 3D convolution model. You would want to use TimeDistributed while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [18]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from tensorflow.keras.layers import LeakyReLU
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau , EarlyStopping
from keras import optimizers
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

Let us create the train_generator and the val_generator which will be used in .fit_generator.

In [25]:
num_classes=5
model = Sequential()
model.add(Conv3D(32,(3,3,3), strides=(1,1,1), padding='same', 
          input_shape=(15,100,100,3)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling3D(pool_size=(2,2,1),strides=(2,2,1)))

model.add(Conv3D(64, (3,3,3), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(Conv3D(64, (3,3, 3)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling3D(pool_size=(2, 2,2)))
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# used 'SGD' optimizer 

In [26]:
optimiser = 'sgd' #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_3 (Conv3D)            (None, 15, 100, 100, 32)  2624      
_________________________________________________________________
batch_normalization_3 (Batch (None, 15, 100, 100, 32)  128       
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 15, 100, 100, 32)  0         
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 7, 50, 100, 32)    0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 7, 50, 100, 64)    55360     
_________________________________________________________________
batch_normalization_4 (Batch (None, 7, 50, 100, 64)    256       
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 7, 50, 100, 64)   

# Batch size = 32

In [27]:
batch_size=32
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [29]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR =  ReduceLROnPlateau(monitor="val_loss",
    patience=10,
    cooldown=0,
    min_lr=0.01) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [30]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

# Fit the model

In [31]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=30, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Source path =  /content/Project_data/train ; batch size = 32
Epoch 1/30
21/21 [==============================] - 44s 2s/step - loss: 42.1568 - categorical_accuracy: 0.2132 - val_loss: 75.4596 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to model_init_2021-03-2911_14_22.102284/model-00001-22.28024-0.23831-75.45963-0.21000.h5
Epoch 2/30
21/21 [==============================] - 27s 1s/step - loss: 1.4784 - categorical_accuracy: 0.3920 - val_loss: 21.5812 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2021-03-2911_14_22.102284/model-00002-1.56848-0.39545-21.58123-0.25000.h5
Epoch 3/30
21/21 [==============================] - 24s 1s/step - loss: 1.3342 - categorical_accuracy: 0.4072 - val_loss: 12.6081 - val_categorical_accuracy: 0.1875

Epoch 00003: saving model to model_init_2021-03-2911_14_22.102284/model-00003-1.40879-0.40515-12.60811-0.18750.h5
Epoch 4/30
21/21 [==============================] - 22s 1s/step - loss: 1.4069 - categorical_accu

## The model obtained has a training accuracy = 98.30 and validation accuract = 87.5

## The link for h5 file: https://drive.google.com/file/d/1wA3ypBbYKWtqlRIQT-dQy_wPg71p_Pv1/view?usp=sharing